In [1]:
%matplotlib
from pathlib import Path
#
from matplotlib import pyplot as plt
from PIL import Image
#
import numpy as np
import cv2

Using matplotlib backend: Qt5Agg


## Superimpose pred thresholded

In [189]:
fold = 'C1-P5' #  # C2-P1 # complet pr ep20 C1-P5
nb_epochs = 20
frame_nb = 199
thresh_level = 23
addr = f'epxx_20x_thresh/ep{nb_epochs}_20x/{fold}/frame{frame_nb}'
img = cv2.imread(addr + '.png')
img_bf = cv2.imread(addr + '.tiff')
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)             # self.img_mask
ret, thresh = cv2.threshold(imgray, thresh_level, 255, 0)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
img = cv2.drawContours( img_bf, contours, -1, (255, 255, 255), cv2.FILLED ) #  [contours[6]]
plt.imshow(img)

In [191]:
img_bf = cv2.imread(addr + '.tiff')
img = img_bf
nbcells_tot = 0
for i,c in enumerate(contours):
    s = cv2.contourArea(c)
    area0 = 100
    step = 2000000
    if area0 <  s  < area0+step:
        nbcells = int(s*0.045)
        nbcells_tot += nbcells
        print(f'for {i} area is {s}, estimated number of cells is {nbcells}')
        img = cv2.drawContours( img, [c], -1, (255, 255, 255), cv2.FILLED ) #  [contours[6]]
print(f"nbcell tot = {nbcells_tot}")
plt.imshow(img)

for 57 area is 209229.5, estimated number of cells is 9415
for 112 area is 104.0, estimated number of cells is 4
for 185 area is 131.0, estimated number of cells is 5
for 216 area is 243.5, estimated number of cells is 10
for 226 area is 144.0, estimated number of cells is 6
for 821 area is 109.0, estimated number of cells is 4
nbcell tot = 9444


## improve training with C2-P1 pred

In [110]:
addr_extr = '/media/sdata/docs/boulot/tests/improving_20x_training_set/ep20_20x_test_movie_upload_image-C2-P1-BF_17-11-2020-13-05'
ext = 'png'

frame_nb = 190
thresh_level = 31
addr = Path(addr_extr) / f'frame{frame_nb}.{ext}'

In [111]:
img_black = np.zeros( ( 512,512 ), np.uint8  )
img = cv2.imread(str(addr))
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)             # self.img_mask
ret, thresh = cv2.threshold(imgray, thresh_level, 255, 0)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
img = cv2.drawContours( img_black, contours, -1, (255, 255, 255), cv2.FILLED )
img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)  # back to color

In [112]:
## Remove axes, remove axis
fig = plt.figure()
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)

ax.imshow(img)

## thresholding on C2-P1

In [19]:
addr_extr = '/home/meglio/Bureau/git/Tools/unet/training_sets/training_20x/masks'
ext = 'tiff'
frame_nb = 190
thresh_level = 33
addr = Path(addr_extr) / f'{frame_nb}.{ext}'

In [20]:
img_black = np.zeros( ( 512,512 ), np.uint8  )
img = cv2.imread(str(addr), -1)
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)             # self.img_mask
ret, thresh = cv2.threshold(imgray, thresh_level, 255, 0)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
img_cntrs = cv2.drawContours( img, contours, -1, (255, 255, 255), cv2.FILLED )
#img_cntrs = cv2.cvtColor(img_cntrs,cv2.COLOR_GRAY2RGB)  # back to color

In [21]:
## Remove axes, remove axis
fig = plt.figure()
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
fig.add_axes(ax)

ax.imshow(img_cntrs)

## adaptive thresholding on C2-P1

In [104]:
def thresholdings(imgray,thresh_level):
    '''
    '''
    ret,th1a = cv2.threshold(imgray, thresh_level, 255, 0)
    ret,th1b = cv2.threshold(img, thresh_level, 255, cv2.THRESH_BINARY)
    ret,th1c = cv2.threshold(img, thresh_level, 255, cv2.THRESH_TRUNC)
    ret,th1d = cv2.threshold(img, thresh_level, 255, cv2.THRESH_TOZERO)
    ret,th1e = cv2.threshold(img, thresh_level, 255, cv2.THRESH_TOZERO_INV)
    th1 = [th1a,th1b,th1c,th1d,th1e]
    th2 = cv2.adaptiveThreshold(imgray,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
    th3 = cv2.adaptiveThreshold(imgray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    return th1, th2, th3

In [105]:
from skimage.restoration import denoise_tv_chambolle
addr_extr = '/home/meglio/Bureau/git/Tools/unet/training_sets/training_20x/masks'
ext = 'tiff'
frame_nb = 190
thresh_level = 33
addr = Path(addr_extr) / f'{frame_nb}.{ext}'

In [192]:
img_black = np.zeros( ( 512,512 ), np.uint8  )
img = cv2.imread(str(addr), -1)
img = denoise_tv_chambolle(img, weight=0.03)
imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)             # self.img_mask
#imgray = denoise_tv_chambolle(imgray, weight=0.03)
th1, th2, th3 = thresholdings(imgray,thresh_level)
contours, hierarchy = cv2.findContours(th2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
img_cntrs = cv2.drawContours( img, contours, -1, (255, 255, 255), cv2.FILLED )
#img_cntrs = cv2.cvtColor(img_cntrs,cv2.COLOR_GRAY2RGB)  # back to color

AttributeError: 'NoneType' object has no attribute 'dtype'

In [108]:
## Remove axes, remove axis
#plt.title()
fig = plt.figure()
ax = plt.Axes(fig, [0., 0., 1., 1.])
ax.set_axis_off()
#ax.set_title(f'figure {frame_nb}')
fig.add_axes(ax)
ax.imshow(img_cntrs)

In [177]:
24/526.5

0.045584045584045586